### Imports + graph creation

In [ ]:
import csv
from datetime import timedelta
import gurobipy as gp 
from gurobipy import GRB
import networkx as nx
import random
import io
import time
import contextlib
import matplotlib.pyplot as plt
import inspect
from collections import defaultdict

from models import simple_mpc, mpc_duration_constr, lazy, column_generation, column_generation2, column_generation3, cg_heuristics
from helper import Service, hhmm2mins, mins2hhmm, fetch_data, fetch_data_by_rake, draw_graph_with_edges, node_legal, no_overlap, create_duty_graph, extract_nodes, generate_paths, roster_statistics, solution_verify, restricted_linear_program, generate_initial_feasible_duties_random_from_services, generate_new_column, mip, restricted_linear_program_for_heuristic, count_overlaps


In [ ]:
import importlib
import models
import helper

importlib.reload(models)
importlib.reload(helper)

from models import simple_mpc, mpc_duration_constr, lazy, column_generation, column_generation2, column_generation3, cg_heuristics
from helper import Service, hhmm2mins, mins2hhmm, fetch_data, fetch_data_by_rake, draw_graph_with_edges, node_legal, no_overlap, create_duty_graph, extract_nodes, generate_paths, roster_statistics, solution_verify, restricted_linear_program, generate_initial_feasible_duties_random_from_services, generate_new_column, mip, restricted_linear_program_for_heuristic, count_overlaps

In [ ]:
# services, service_dict = fetch_data('./StepBackServices.csv')
services, service_dict = fetch_data_by_rake('./StepBackServices.csv', partial=True, rakes=20)
graph = create_duty_graph(services)
print(graph)

### Main

In [ ]:
repeat =True
while repeat:
    num = int(input("""\nEnter the Model you'd like to run (integer): 
                    
                    1. Simple MPC
                    2. MPC with Duration Constraint
                    3. Lazy
                    4. Column Generation
                    5. Column Generation with IP with heuristic
                    6. Column Generation with bf with duty duration
                    
                    Expecting input: """))
    if num == 1:
        print("\nSimple MPC Model")
        duties, duty_count = simple_mpc(graph, service_dict, show_logs = False, show_duties = False, show_roster_stats = True)
        # print(duties)
        # print(duty_count)
    elif num == 2:
        print("\nMPC with Duration Constraints")
        duties, duty_count = mpc_duration_constr(graph, service_dict, time_limit = 60, show_logs = True, show_duties = False, show_roster_stats = True)
        # print(duties)
        # print(duty_count)
    elif num == 3:
        print("\nLazy Model")
        duties, duty_count = lazy(graph, service_dict, show_logs = False, max_duty_duration=6*60, lazy_iterations =100, show_lazy_updates_every = 10, show_duties = False, show_roster_stats = True)
    elif num == 4:
        print("\nColumn Generation Model")
        # duties, selected_duties, obj = column_generation(graph, service_dict, init_column_generator = "random", pricing_method = "bellman ford", iterations = 10, verbose = True)
        mpc_sol, column_pool, duties, selected_duties, obj = column_generation(graph, service_dict, init_column_generator = "mpc", mpc_timeout = 20,pricing_method = "topological sort", iterations = 500, verbose = True)
        # selected_duties -  a list of tuples (var_name, var.x)
        roster_statistics(duties.values(), service_dict)
    # elif num == 5:
    #     print("\nColumn Generation Model 2 with IP with heuristic")
    #     mpc_sol, current_duties, final_duties, selected_duties, obj = column_generation2(graph, service_dict, init_column_generator = "mpc", mpc_timeout=30, pricing_method = "topological sort", iterations = 10000, verbose = True)
    #     roster_statistics(final_duties.values(), service_dict)
    #     print("\n================================================================\n")
    #     final_final_duties, all_duties = cg_heuristics(graph, service_dict, current_duties = list(final_duties.values()), threshold=0.8)
    elif num == 5:
    # 5. Column Generation + Heuristic → write results + timings to file
        with open('output.txt', 'w') as f:
            # 1. MPC solution (and LP obj) timing
            f.write("1. MPC solution from column_generation2:\n")
            t1_start = time.time()
            mpc_sol, current_duties, final_duties, selected_duties, obj = column_generation2(
                graph,
                service_dict,
                init_column_generator="mpc",
                mpc_timeout=30,
                pricing_method="topological sort",
                iterations=10000,
                verbose=False
            )
            t1_end = time.time()
            f.write(f"{mpc_sol}\n\n")
            f.write("   LP objective from column_generation2:\n")
            f.write(f"{obj}\n")
            f.write(f"   [Time taken: {t1_end - t1_start:.2f}]\n\n")

            # 2. Roster statistics timing
            f.write("2. Roster statistics on LP duties:\n")
            buf_stats = io.StringIO()
            with contextlib.redirect_stdout(buf_stats):
                roster_statistics(final_duties.values(), service_dict)
            t2_end = time.time()
            f.write(buf_stats.getvalue())
            

            # 4. Final objective from cg_heuristics
            # f.write("4. Final objective after cg_heuristics:\n")
            # cg_heuristics must now return (duties, all_duties, final_obj)
            final_final_duties, all_duties = cg_heuristics(
                graph,
                service_dict,
                current_duties=list(final_duties.values()),
                threshold=0.8
            )
            # f.write(f"{final_obj}\n\n")

            # # 5. Overlap count and duties list
            # f.write("5. Overlap count and selected duty list (tt):\n")
            # buf_ol = io.StringIO()
            # with contextlib.redirect_stdout(buf_ol):
            #     tt = [ all_duties[i] for i in final_final_duties ]
            #     count_overlaps(tt, service_dict)
            #     print("Selected duties (tt):", tt)
            # f.write(buf_ol.getvalue())

        print("✅ All results written to output.txt")
    elif num == 6:
        print("\nColumn Generation Model 2 with bf with duty duration")
        mpc_sol, current_duties, final_duties, selected_duties, obj = column_generation2(graph, service_dict, init_column_generator = "mpc", mpc_timeout=30, pricing_method = "bf_duration_constr", iterations = 10000, verbose = True)
        roster_statistics(final_duties.values(), service_dict)
        print("\n=================================================================\n")
        # CHECK FOR BF
        final_final_duties, all_duties = cg_heuristics(graph, service_dict, current_duties = list(final_duties.values()), threshold=0.6)
    else:
        print("\nInvalid Input")

    repeat =False

    # repeat = input("\nWould you like to run another model? (y/n): ").lower() == 'y'


In [ ]:
tt = []
for i in final_final_duties:
    tt.append(all_duties[i])

count_overlaps(tt, services)
print(tt)

### Getting an integer solution from column generation fractional solution

In [ ]:
sorted_duties = sorted(selected_duties, key=lambda x: x[1], reverse=True)

print(sorted_duties)

In [ ]:
final = []
for var_name, var_val in sorted_duties:
    #duties is a dictionary - (var_name, duty (a list of service_nums))
    final.append(duties[var_name])
    if solution_verify(service_dict.values(), final, verbose =False):
        break

print("Required duties: ", len(final))
print("Total duties: ", len(selected_duties))
roster_statistics(final, service_dict)

In [ ]:
obj, selected_duties = mip(service_dict, column_pool, show_solutions = True, show_objective = True, warm = mpc_sol)

In [ ]:
final_2 = []
for var_name, var_val in selected_duties:
    final_2.append(duties[var_name])
    

print("Required duties: ", len(final))
print("Total duties: ", len(selected_duties))
roster_statistics(final, service_dict)

### SPPRC DP Implementation

In [ ]:
import networkx as nx

# Create a directed graph (use nx.Graph() for an undirected graph)
G = nx.DiGraph()

# Add nodes with the "service_time" attribute
nodes = {
    -2 : {"service_time": 0},
    -1 : {"service_time": 0},
}
for node in range(1,6):
    nodes[node]= {"service_time" :2}

# print (nodes)

G.add_nodes_from(nodes.items())

# Add edges with the "cost" attribute
edges = [
    (1, 2, {"cost": -2}),
    (2, 3, {"cost": -5}),
    (3, 4, {"cost": -4}),
    (4, 5, {"cost": -5}),
    (1, 3, {"cost": -6})
    # (5, -1, {"cost": -4})
]

edge_dict = {
    (1, 2) : {"cost": -2},
    (2, 3) : {"cost": -5},
    (3, 4) : {"cost": -4},
    (4, 5) : {"cost": -5},
    (1, 3) : {"cost": -6}
    # (5, -1) : {"cost": -4}
}


for node in range(1,6):
    edges.append( (-2 , node, {"cost" : 0} ))
    edges.append( (node, -1, {"cost" : -4} ))

    edge_dict[(-2, node)] = {"cost" : 0}
    edge_dict[(node, -1)] = {"cost" : -4}

edges.append( (-2, -1, {"cost" : 0} ))
edge_dict[(-2, -1)] = {"cost" : 0}

G.add_edges_from(edges)

# Print all node attributes
# print("Nodes with attributes:")
# for node, attr in G.nodes(data=True):
#     print(f"{node}: {attr}")

# Print all edges with attributes
print("\nEdges with attributes:")
for u, v, attr in G.edges(data=True):
    print(f"Edge ({u} -> {v}): {attr}")

# # # Access specific attributes
# # print("\nService time at node A:", G.nodes["A"]["service_time"])
# # print("Cost of edge A -> B:", G["A"]["B"]["cost"])

# print(edge_dict)




In [ ]:
from collections import defaultdict
from collections import deque

dp_dict = defaultdict(dict)

topo_order = list(nx.topological_sort(G))

for time in range(7):
    dp_dict[-2][time] = (0, None)

for node in topo_order:
    for time in range(7):
        best_pred = None
        best = 0
        for pred in G.predecessors(node):
            if G.nodes[pred]["service_time"] > time:
                continue
            time_range = max(0, time - G.nodes[pred]["service_time"]) #node to pred change
            for t in range(time_range + 1):
                current = dp_dict[pred][t][0] + edge_dict [(pred, node)]["cost"]
                # print("Pred: ", pred, ", Time: ", time, ", T: ",t , "Current: ",current, "dp_dict", dp_dict[pred][t])
                if current <= best:
                    best = current
                    best_pred = pred
        dp_dict[node][time] = (best, best_pred)
    print(dp_dict)

remaining_time = 6 
current = -1
spprc = dp_dict[current][remaining_time][0]

path = deque()

while current != -2: # None to -2
    path.appendleft(current)
    # print(current, remaining_time)
    pred = dp_dict[current][remaining_time][1]
    remaining_time -= G.nodes[pred]["service_time"] #current to pred
    # remaining_time = max(0, remaining_time)
    current =pred
    # print("Shortest Path: ", path)  

print("Shortest Path Length: ", spprc)
print("Shortest Path: ", path)    
# dp_dict

In [ ]:
for pred in G.predecessors(-2):
    print("hI")

### Column Generation using SPPRC DP

In [ ]:
mpc_sol, column_pool, duties, selected_duties, obj = column_generation(graph, service_dict, init_column_generator = "mpc", mpc_timeout = 60,pricing_method = "dp", iterations = 2, verbose = True)

In [ ]:
mpc_sol, column_pool, duties, selected_duties, obj = column_generation(graph, service_dict, init_column_generator = "mpc", mpc_timeout = 20,pricing_method = "dp", iterations = 1, verbose = True)

### Column generation with IP column generator

In [ ]:
mpc_sol, column_pool, duties, selected_duties, obj = column_generation3(graph, service_dict, init_column_generator = "mpc", mpc_timeout = 30,pricing_method = "ip", iterations = 2, verbose = True)

### Heuristics for column generation integer solution

In [ ]:
def cg_heuristics():
    pass

In [ ]:
selected_duties

### Plotting a graph

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Define edges with (node1, node2, cost, time)
edges = [
    ("s", 1 , 50 , 30),
    ("s", 2, 43, 91),
    ("s", 3, 5, 56),
    (1,5,55,38),
    (5,1,55,38),
    (2,6,35,51),
    (3,6,19,66),
    (3,2,28,42),
    (5,6,31,42),
    (6,5,31,42),
    (5,"t",47,15),
    ("t", 5, 47, 15),
    (6,"t", 85, 5)
]

# Create a directed graph
G = nx.DiGraph()

# Add edges with attributes
for u, v, cost, time in edges:
    G.add_edge(u, v, cost=cost, time=time)

# Define node positions
pos = nx.spring_layout(G, seed=42)  # Layout for visualization

# Draw the graph
plt.figure(figsize=(8, 6))
nx.draw(G, pos, with_labels=True, node_color='lightblue', edge_color='gray', node_size=2000, font_size=12, font_weight='bold')

# Draw edge labels (cost and time)
edge_labels = {(u, v): f"Cost: {d['cost']}, Time: {d['time']}" for u, v, d in G.edges(data=True)}
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=10, label_pos=0.3)

plt.title("Graph Visualization with Edge Attributes (Cost & Time)")
plt.show()


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Define edges with (node1, node2, cost, time)
edges = [
    ("s", 1 , 50 , 30),
    ("s", 2, 43, 91),
    ("s", 3, 5, 56),
    (1,5,55,38),
    (5,1,55,38),
    (2,6,35,51),
    (3,6,19,66),
    (3,2,28,42),
    (5,6,31,42),
    (6,5,31,42),
    (5,"t",47,15),
    ("t", 5, 47, 15),
    (6,"t", 85, 5)
]

# Create a directed graph
G = nx.DiGraph()

# Add edges with attributes
for u, v, cost, time in edges:
    G.add_edge(u, v, cost=cost, time=time)

# Define node positions (3 in each column)
pos = {
    "s": (0,1),
    1: (1, 2), 2: (1, 1), 3: (1, 0),  # Left column
    5: (2, 1), 6: (2, 0),
    "t": (3,1)   # Right column
}

# Draw the graph
plt.figure(figsize=(6, 6))
nx.draw(G, pos, with_labels=True, node_color='lightblue', edge_color='gray', node_size=2000, font_size=12, font_weight='bold')

# Draw edge labels (cost and time)
edge_labels = {(u, v): f"C: {d['cost']}, T: {d['time']}" for u, v, d in G.edges(data=True)}
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=10, label_pos=0.3)

plt.title("Graph with Vertically Aligned Nodes")
plt.show()


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Define edges with (node1, node2, cost, time)
edges = [
    ("s", 1 , 50 , 30),
    ("s", 2, 43, 91),
    ("s", 3, 5, 56),
    (1,5,55,38),
    (5,1,55,38),
    (2,6,35,51),
    (3,6,19,66),
    (3,2,28,42),
    (5,6,31,42),
    (6,5,31,42),
    (5,"t",47,15),
    ("t", 5, 47, 15),
    (6,"t", 85, 5)
]

# Create a directed graph
G = nx.DiGraph()

# Add edges with attributes
for u, v, cost, time in edges:
    G.add_edge(u, v, cost=cost, time=time)

# Define node positions
pos = {
    "s": (0,1),
    1: (1, 2), 2: (1, 1), 3: (1, 0),
    5: (2, 1), 6: (2, 0),
    "t": (3,1)
}

# Compute shortest paths and their costs
shortest_paths = {}
shortest_costs = {}
shortest_edges = []

for node in G.nodes:
    if node != "t":  # Exclude target node itself
        try:
            path = nx.shortest_path(G, source=node, target="t", weight="cost")
            cost = nx.shortest_path_length(G, source=node, target="t", weight="cost")
            shortest_paths[node] = path
            shortest_costs[node] = cost
            shortest_edges.extend(list(zip(path[:-1], path[1:])))
        except nx.NetworkXNoPath:
            shortest_paths[node] = None  # No path available
            shortest_costs[node] = None  # No cost available

# Draw the graph
plt.figure(figsize=(6, 6))
nx.draw(G, pos, with_labels=True, node_color='lightblue', edge_color='gray', node_size=2000, font_size=12, font_weight='bold')

# Draw edge labels (cost)
edge_labels = {(u, v): f"C: {d['cost']}" for u, v, d in G.edges(data=True)}
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=10, label_pos=0.3)

# Highlight shortest paths in red
# nx.draw_networkx_edges(G, pos, edgelist=shortest_edges, edge_color='red', width=2)

plt.title("Shortest Paths to Node 't' (Based on Cost)")
plt.show()

# Print shortest paths and costs
print("Shortest Paths to 't' with Costs:")
for node in shortest_paths:
    if shortest_paths[node] is not None:
        print(f"From {node} → Path: {shortest_paths[node]}, Cost: {shortest_costs[node]}")
    else:
        print(f"From {node} → No path to 't'")


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Define edges with (node1, node2, cost, time)
edges = [
    ("s", 1 , 50 , 30),
    ("s", 2, 43, 91),
    ("s", 3, 5, 56),
    (1,5,55,38),
    (5,1,55,38),
    (2,6,35,51),
    (3,6,19,66),
    (3,2,28,42),
    (5,6,31,42),
    (6,5,31,42),
    (5,"t",47,15),
    ("t", 5, 47, 15),
    (6,"t", 85, 5)
]

# Create a directed graph
G = nx.DiGraph()

# Add edges with attributes
for u, v, cost, time in edges:
    G.add_edge(u, v, cost=cost, time=time)

# Define node positions
pos = {
    "s": (0,1),
    1: (1, 2), 2: (1, 1), 3: (1, 0),
    5: (2, 1), 6: (2, 0),
    "t": (3,1)
}

# 1️⃣ Shortest paths from all nodes to "t" based on "cost"
shortest_paths_to_t_cost = {}
shortest_costs_to_t_cost = {}

for node in G.nodes:
    if node != "t":  
        try:
            path = nx.shortest_path(G, source=node, target="t", weight="cost")
            cost = nx.shortest_path_length(G, source=node, target="t", weight="cost")
            shortest_paths_to_t_cost[node] = path
            shortest_costs_to_t_cost[node] = cost
        except nx.NetworkXNoPath:
            shortest_paths_to_t_cost[node] = None
            shortest_costs_to_t_cost[node] = None

# 2️⃣ Shortest paths from "s" to all nodes based on "time"
shortest_paths_from_s_time = {}
shortest_costs_from_s_time = {}

for node in G.nodes:
    if node != "s":  
        try:
            path = nx.shortest_path(G, source="s", target=node, weight="time")
            time = nx.shortest_path_length(G, source="s", target=node, weight="time")
            shortest_paths_from_s_time[node] = path
            shortest_costs_from_s_time[node] = time
        except nx.NetworkXNoPath:
            shortest_paths_from_s_time[node] = None
            shortest_costs_from_s_time[node] = None

# 3️⃣ Shortest paths from all nodes to "t" based on "time"
shortest_paths_to_t_time = {}
shortest_costs_to_t_time = {}

for node in G.nodes:
    if node != "t":  
        try:
            path = nx.shortest_path(G, source=node, target="t", weight="time")
            time = nx.shortest_path_length(G, source=node, target="t", weight="time")
            shortest_paths_to_t_time[node] = path
            shortest_costs_to_t_time[node] = time
        except nx.NetworkXNoPath:
            shortest_paths_to_t_time[node] = None
            shortest_costs_to_t_time[node] = None

# Draw the graph
plt.figure(figsize=(6, 6))
nx.draw(G, pos, with_labels=True, node_color='lightblue', edge_color='gray', node_size=2000, font_size=12, font_weight='bold')

# Draw edge labels (cost and time)
edge_labels = {(u, v): f"C: {d['cost']}, T: {d['time']}" for u, v, d in G.edges(data=True)}
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=10, label_pos=0.3)

plt.title("Graph with Shortest Paths (Cost & Time)")
plt.show()

# Print shortest paths to "t" based on cost
print("\nShortest Paths from All Nodes to 't' (Using Cost):")
for node in shortest_paths_to_t_cost:
    if shortest_paths_to_t_cost[node] is not None:
        print(f"From {node} → 't': Path = {shortest_paths_to_t_cost[node]}, Cost = {shortest_costs_to_t_cost[node]}")
    else:
        print(f"From {node} → 't': No path available")

# Print shortest paths from "s" based on time
print("\nShortest Paths from 's' to All Nodes (Using Time):")
for node in shortest_paths_from_s_time:
    if shortest_paths_from_s_time[node] is not None:
        print(f"From 's' → {node}: Path = {shortest_paths_from_s_time[node]}, Time = {shortest_costs_from_s_time[node]}")
    else:
        print(f"From 's' → {node}: No path available")

# Print shortest paths to "t" based on time
print("\nShortest Paths from All Nodes to 't' (Using Time):")
for node in shortest_paths_to_t_time:
    if shortest_paths_to_t_time[node] is not None:
        print(f"From {node} → 't': Path = {shortest_paths_to_t_time[node]}, Time = {shortest_costs_to_t_time[node]}")
    else:
        print(f"From {node} → 't': No path available")
